In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import random
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Define constants
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 2
DATA_DIR = 'PetImages'

print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

# Data preparation
def prepare_data():
    """
    Prepare training and validation data from PetImages directory structure
    """
    # Create data generators with validation split
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2  # Use 20% for validation
    )

    # Load training data
    train_generator = train_datagen.flow_from_directory(
        DATA_DIR,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='training',
        seed=42
    )

    # Load validation data (no augmentation)
    validation_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    validation_generator = validation_datagen.flow_from_directory(
        DATA_DIR,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='validation',
        seed=42
    )

    print(f"Training samples: {train_generator.samples}")
    print(f"Validation samples: {validation_generator.samples}")
    print(f"Class indices: {train_generator.class_indices}")

    return train_generator, validation_generator

# Build transfer learning model
def build_model():
    """
    Build a transfer learning model using VGG16 as base
    """
    # Load pre-trained VGG16 model without top layers
    base_model = VGG16(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )

    # Freeze the convolutional base initially
    base_model.trainable = False

    # Create new model with GlobalAveragePooling2D for better performance
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model, base_model

def plot_training_history(history):
    """
    Plot training and validation accuracy/loss
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()

    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()

    plt.tight_layout()
    plt.show()

def evaluate_model(model, validation_generator):
    """
    Evaluate model and show detailed metrics
    """
    # Get predictions
    validation_generator.reset()
    predictions = model.predict(validation_generator, verbose=1)
    predicted_classes = (predictions > 0.5).astype(int).flatten()

    # Get true labels
    true_classes = validation_generator.classes
    class_labels = list(validation_generator.class_indices.keys())

    # Classification report
    print("\nClassification Report:")
    print(classification_report(true_classes, predicted_classes, target_names=class_labels))

    # Confusion matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels, yticklabels=class_labels)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    return predictions, predicted_classes

def test_model_on_samples(model_path='cats_vs_dogs_transfer_learning_model.h5', num_samples=6):
    """
    Test the trained model on random sample images
    """
    # Load the trained model
    try:
        model = load_model(model_path)
        print(f"Model loaded from {model_path}")
    except:
        print("Model not found. Please train the model first.")
        return

    # Get random sample images
    cat_dir = os.path.join(DATA_DIR, 'Cat')
    dog_dir = os.path.join(DATA_DIR, 'Dog')

    cat_images = [f for f in os.listdir(cat_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    dog_images = [f for f in os.listdir(dog_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Select random samples
    sample_cats = random.sample(cat_images, min(num_samples//2, len(cat_images)))
    sample_dogs = random.sample(dog_images, min(num_samples//2, len(dog_images)))

    fig, axes = plt.subplots(2, num_samples//2, figsize=(15, 8))

    # Test cat images
    for i, img_name in enumerate(sample_cats):
        img_path = os.path.join(cat_dir, img_name)

        # Load and preprocess image
        try:
            img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array /= 255.0

            # Make prediction
            prediction = model.predict(img_array)[0][0]
            predicted_class = "Dog" if prediction > 0.5 else "Cat"
            confidence = prediction if prediction > 0.5 else 1 - prediction

            # Display image
            axes[0, i].imshow(img)
            axes[0, i].set_title(f'True: Cat\nPred: {predicted_class}\nConf: {confidence:.3f}')
            axes[0, i].axis('off')

        except Exception as e:
            axes[0, i].text(0.5, 0.5, 'Image Error', ha='center', va='center')
            axes[0, i].axis('off')

    # Test dog images
    for i, img_name in enumerate(sample_dogs):
        img_path = os.path.join(dog_dir, img_name)

        # Load and preprocess image
        try:
            img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array /= 255.0

            # Make prediction
            prediction = model.predict(img_array)[0][0]
            predicted_class = "Dog" if prediction > 0.5 else "Cat"
            confidence = prediction if prediction > 0.5 else 1 - prediction

            # Display image
            axes[1, i].imshow(img)
            axes[1, i].set_title(f'True: Dog\nPred: {predicted_class}\nConf: {confidence:.3f}')
            axes[1, i].axis('off')

        except Exception as e:
            axes[1, i].text(0.5, 0.5, 'Image Error', ha='center', va='center')
            axes[1, i].axis('off')

    plt.tight_layout()
    plt.show()

# Main execution
def main():
    # Prepare data
    print("Preparing data...")
    train_generator, validation_generator = prepare_data()

    # Build model
    print("\nBuilding model...")
    model, base_model = build_model()
    model.summary()

    # Define callbacks
    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ReduceLROnPlateau(factor=0.2, patience=3, min_lr=0.0001)
    ]

    # Train model (initial training with frozen base)
    print("\nTraining model (frozen base)...")
    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=validation_generator,
        callbacks=callbacks,
        verbose=1
    )

    # Fine-tuning: unfreeze some layers of the base model
    print("\nFine-tuning model...")
    base_model.trainable = True

    # Fine-tune from this layer onwards
    fine_tune_at = 100

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # Use a lower learning rate for fine-tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001/10),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Continue training for fine-tuning
    fine_tune_epochs = 10
    total_epochs = len(history.history['loss']) + fine_tune_epochs

    history_fine = model.fit(
        train_generator,
        epochs=total_epochs,
        initial_epoch=len(history.history['loss']),
        validation_data=validation_generator,
        callbacks=callbacks,
        verbose=1
    )

    # Combine histories
    for key in history.history:
        history.history[key].extend(history_fine.history[key])

    # Plot training history
    plot_training_history(history)

    # Evaluate model
    print("\nEvaluating model...")
    val_loss, val_accuracy = model.evaluate(validation_generator, verbose=1)
    print(f"Final Validation accuracy: {val_accuracy:.4f}")
    print(f"Final Validation loss: {val_loss:.4f}")

    # Detailed evaluation
    predictions, predicted_classes = evaluate_model(model, validation_generator)

    # Save model
    model.save('cats_vs_dogs_transfer_learning_model.h5')
    print("\nModel saved as 'cats_vs_dogs_transfer_learning_model.h5'")

    # Test on sample images
    print("\nTesting on sample images...")
    test_model_on_samples()

    return history, model

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'tensorflow.keras'